### Previsão de Nota do IMDB

In [19]:
import pandas as pd
import numpy as np

df = pd.read_csv("dataset_tratado.csv")

In [38]:

from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.pipeline import Pipeline
from sklearn .preprocessing import StandardScaler
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV 
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score, root_mean_squared_error
import joblib


O problema que estamos trabalhando é de regressão porque termina com a previsão de um valor numérico e contínuo e não de uma categoria.

Modelos escolhidos:

- **Modelo 1:** Regressão Linear
- **Modelo 2:** Árvore de Decisão
- **Modelo 3:** Random Forest
- **Modelo 4:** Gradient Boosting
- **Modelo 5:** k-Nearest Neighbors (KNN)

Variáveis escolhidas:

- Meta_score: Representa a nota da crítica. É um forte indicador da qualidade do filme e teve uma correlação de 0.29 com a nota do IMDB

- No_of_Votes e Gross: Variáveis servem como indicadores de popularidade e sucesso econômico. Filmes com grande engajamento de público e alto faturamento geralmente são mais conhecidos e tendem a ter uma nota mais estabilizada.

- Released_Year e Runtime: O ano de lançamento ajuda a identificar tendências históricas e a duração do filme pode influenciar a forma como ele é avaliado pelo público, além do Runtime ter correlação de 0.24 com a nota do IMDB.

- Genre: Alguns dos gêneros apresentaram alta correlação com a nota do IMDB, possibilitando que isso ajude na previsão.

- Certificate: Algumas classificações tem notas mais baixas que outras o que ajuda na avaliação do modelo.

Medidas de Performance escolhida:

- As medidas escolhidas foram R2 e o RMSE, O R2 mede a capacidade de o modelo explicar a variabilidade dos dados, enquanto o RMSE quantifica a precisão das previsões de erro, mostrando assim tanto o acerto quanto o erro do modelo.

#### Separação entre variáveis preditoras e alvo

In [21]:
# As colunas sem as linhas faltantes tiveram um desempenho melhor do que as colunas imputadas com a mediana
df = df.dropna(subset=['Released_Year'])
df = df.dropna(subset=['Gross_original'])
df = df.dropna(subset=['Meta_score_original'])

X = df[['Meta_score_original','No_of_Votes','Runtime','Gross_original', 'Released_Year','Certificate', 'Action', 'Adventure',
       'Animation', 'Biography', 'Comedy', 'Crime', 'Drama', 'Family',
       'Fantasy', 'Film-Noir', 'History', 'Horror', 'Music', 'Musical',
       'Mystery', 'Romance', 'Sci-Fi', 'Sport', 'Thriller', 'War', 'Western']]
X = pd.get_dummies(X)
y = df['IMDB_Rating']

#### Separação treino-teste

In [22]:
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size = 0.2, random_state=3)

#### Avaliação com k-fold (GridSearch)

In [25]:
models_params = {
    'Linear Regression': {
        'model': Pipeline([
            ('scaler', StandardScaler()),
            ('clf', LinearRegression())
        ]),
        'params': {
            'clf__fit_intercept': [True, False],
        }
    },
    'Decision Tree': {
        'model': Pipeline([
            ('scaler', StandardScaler()),
            ('clf', DecisionTreeRegressor(random_state=42))
        ]),
        'params': {
            'clf__max_depth': [3, 5, 10, None],
            'clf__min_samples_split': [2, 5, 10]
        }
    },
    'Random Forest': {
        'model': Pipeline([
            ('scaler', StandardScaler()),
            ('clf', RandomForestRegressor(random_state=42))
        ]),
        'params': {
            'clf__n_estimators': [50, 100, 200],
            'clf__max_depth': [3, 5, 10, None],
            'clf__min_samples_split': [2, 5, 10]
        }
    },
    'Gradient Boosting': {
        'model': Pipeline([
            ('scaler', StandardScaler()),
            ('clf', GradientBoostingRegressor(random_state=42))
        ]),
        'params': {
            'clf__n_estimators': [50, 100, 200],
            'clf__learning_rate': [0.01, 0.1, 0.2],
            'clf__max_depth': [3, 5, 7]
        }
    },
    'KNN': {
        'model': Pipeline([
            ('scaler', StandardScaler()),
            ('clf', KNeighborsRegressor())
        ]),
        'params': {
            'clf__n_neighbors': [3, 5, 7, 9],
            'clf__weights': ['uniform', 'distance']
        }
    }
}

scoring = {
    'r2': 'r2',
    'neg_root_mean_squared_error': 'neg_root_mean_squared_error'
}


results = {}
for name, mp in models_params.items():
    print(f"Treinando: {name}")
    grid = GridSearchCV(
        mp['model'],
        mp['params'],
        cv=7,                  
        scoring=scoring,
        refit='r2',           
        n_jobs=-1
    )
    grid.fit(X_treino, y_treino)
    
    results[name] = {
        'R2 Médio (CV)': round(grid.cv_results_['mean_test_r2'][grid.best_index_], 4),
        'RMSE Médio (CV)': round((-grid.cv_results_['mean_test_neg_root_mean_squared_error'][grid.best_index_])**0.5, 4),
        'Melhores Parâmetros': grid.best_params_
    }
 
# Exibindo resultados
for modelo, res in results.items():
    print(f"\n{modelo}")
    print("R2 Médio (CV):", res['R2 Médio (CV)'])
    print("RMSE Médio (CV):", res['RMSE Médio (CV)'])
    print("Melhores Parâmetros:", res['Melhores Parâmetros'])

Treinando: Linear Regression
Treinando: Decision Tree
Treinando: Random Forest
Treinando: Gradient Boosting
Treinando: KNN

Linear Regression
R2 Médio (CV): 0.5619
RMSE Médio (CV): 0.4317
Melhores Parâmetros: {'clf__fit_intercept': True}

Decision Tree
R2 Médio (CV): 0.4469
RMSE Médio (CV): 0.4604
Melhores Parâmetros: {'clf__max_depth': 5, 'clf__min_samples_split': 5}

Random Forest
R2 Médio (CV): 0.6098
RMSE Médio (CV): 0.4207
Melhores Parâmetros: {'clf__max_depth': None, 'clf__min_samples_split': 2, 'clf__n_estimators': 50}

Gradient Boosting
R2 Médio (CV): 0.6021
RMSE Médio (CV): 0.4226
Melhores Parâmetros: {'clf__learning_rate': 0.1, 'clf__max_depth': 5, 'clf__n_estimators': 50}

KNN
R2 Médio (CV): 0.2441
RMSE Médio (CV): 0.4975
Melhores Parâmetros: {'clf__n_neighbors': 9, 'clf__weights': 'distance'}


### Holdout com os modelos campeões

In [27]:

best_models = {
    'Linear Regression': Pipeline([
        ('scaler', StandardScaler()),
        ('clf', LinearRegression(fit_intercept=True))
    ]),
    'Decision Tree': Pipeline([
        ('scaler', StandardScaler()),
        ('clf', DecisionTreeRegressor(max_depth=5, min_samples_split=5, random_state=42))
    ]),
    'Random Forest': Pipeline([
        ('scaler', StandardScaler()),
        ('clf', RandomForestRegressor(n_estimators=50, max_depth=None, min_samples_split=2, random_state=42))
    ]),
    'Gradient Boosting': Pipeline([
        ('scaler', StandardScaler()),
        ('clf', GradientBoostingRegressor(n_estimators=50, learning_rate=0.1, max_depth=5, random_state=42))
    ]),
    'KNN': Pipeline([
        ('scaler', StandardScaler()),
        ('clf', KNeighborsRegressor(n_neighbors=9, weights='distance'))
    ])
}
 
# DataFrame para armazenar resultados
test_results = {
    'Modelo': [],
    'R2 (teste)': [],
    'RMSE (teste)': []
}
 
# Avaliação dos modelos
for name, model in best_models.items():
    model.fit(X_treino, y_treino)
    
    # Predição
    y_pred = model.predict(X_teste)
    
    # Métricas
    r2 = r2_score(y_teste, y_pred)
    rmse = root_mean_squared_error(y_teste, y_pred)  
    
    # Armazena resultados
    test_results['Modelo'].append(name)
    test_results['R2 (teste)'].append(round(r2, 4))
    test_results['RMSE (teste)'].append(round(rmse, 4))
 
# Exibe resultados
for i in range(len(test_results['Modelo'])):
    print(f"\n{test_results['Modelo'][i]}")
    print("R2 (teste):", test_results['R2 (teste)'][i])
    print("RMSE (teste):", test_results['RMSE (teste)'][i])


Linear Regression
R2 (teste): 0.3663
RMSE (teste): 0.2105

Decision Tree
R2 (teste): 0.3625
RMSE (teste): 0.2112

Random Forest
R2 (teste): 0.5325
RMSE (teste): 0.1808

Gradient Boosting
R2 (teste): 0.5269
RMSE (teste): 0.1819

KNN
R2 (teste): 0.0694
RMSE (teste): 0.2551


### Discussão

- O **Random Forest** foi o modelo com melhor desempenho geral, alcançando o maior R² (0.5325) e o menor RMSE (0.1808), demonstrando ser a abordagem mais precisa e com maior capacidade de explicar a variabilidade dos dados.
- O **Gradient Boosting** apresentou uma performance quase idêntica à do Random Forest, com R² de 0.5269 e RMSE de 0.1819, consolidando-se também como uma solução de alta performance para este problema de regressão.
- A **Regressão Linear** serviu como um *baseline* eficaz, com desempenho intermediário (R² de 0.3663), mostrando que um modelo simples já consegue capturar uma parte significativa dos padrões nos dados.
- A **Árvore de Decisão** teve um desempenho muito próximo ao da Regressão Linear (R² de 0.3625), indicando que, em sua forma individual, não foi capaz de superar a performance do modelo linear.
- O **KNN** apresentou a performance mais fraca, com um R² muito baixo (0.0694) e o maior RMSE (0.2551), o que sugere que a sua abordagem baseada em vizinhança não foi adequada para a estrutura deste conjunto de dados.


- O modelo que mais se aproxima dos dados foi o **Random Forest** com um R2 de 0.5325 acertando 53,25% da variabilidade das notas do IMDb. Indicando que o modelo alcançou um resultado muito bom.  O RMSE indica que a previsão do modelo está apenas 0.18 pontos de distância da nota real do filme. Demonstrando que o modelo não possui muitos contras a se considerar.

# Exemplo

In [32]:
exemplo = {
 'Released_Year': ['1994'],
 'Runtime': [142], # retirando a string 'min' e convertendo o tipo
 'Meta_score_original': [80.0],
 'No_of_Votes': [2343110],
 'Gross_original': [28341469], # retirando as virgulas
 'Action' : [0], 
 'Adventure' : [0],
 'Animation' : [0], 
 'Biography' : [0], 
 'Comedy' : [0], 
 'Crime' : [0], 
 'Drama' : [1], # genero drama verdadeiro
 'Family' : [0],
 'Fantasy' : [0], 
 'Film-Noir' : [0], 
 'History' : [0], 
 'Horror' : [0], 
 'Music' : [0], 
 'Musical' : [0],
 'Mystery' : [0], 
 'Romance' : [0], 
 'Sci-Fi' : [0], 
 'Sport' : [0], 
 'Thriller' : [0], 
 'War' : [0], 
 'Western' : [0], 
 'Certificate_Adultos'  : [True],  # convertendo A para Adultos
 'Certificate_Não Classificado'  : [False], 
 'Certificate_Orientação Parental'  : [False], 
 'Certificate_Universal' : [False]
 }


exemplo = pd.DataFrame(exemplo, columns=X.columns)
exemplo

,Meta_score_original,No_of_Votes,Runtime,Gross_original,Released_Year,Action,Adventure,Animation,Biography,Comedy,...,Romance,Sci-Fi,Sport,Thriller,War,Western,Certificate_Adultos,Certificate_Não Classificado,Certificate_Orientação Parental,Certificate_Universal
0,80.0,2343110,142,28341469,1994,0,0,0,0,0,...,0,0,0,0,0,0,True,False,False,False


In [36]:
best_model = best_models["Random Forest"]
previsao_nota = best_model.predict(exemplo)
print(f'A previsão da nota no IMDB do filme indicado é: {previsao_nota[0]}')

A previsão da nota no IMDB do filme indicado é: 8.742


In [39]:
joblib.dump(best_model, "melhor_modelo_randomforest.pkl")

['melhor_modelo_randomforest.pkl']